In [70]:
import pandas as pd
import datetime
from astropy.time import Time
import spiceypy as spy
import numpy as np
import rebound as rb
import matplotlib.pyplot as plt
from Utils import *
from astroquery.jplhorizons import Horizons
import spiceypy as spy
import plotly.graph_objects as go

In [71]:
fireballs = pd.read_csv('datos/cneos_fireball_data_original.csv', comment='#')
fireballs = fireballs.dropna().reset_index()
fireballs

,index,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),vx,vy,vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
0,0,2023-08-05 19:54:50,19.9N,131.7W,44.0,21.8,3.3,20.4,-6.8,1.719000e+12,3.800
1,1,2023-07-26 03:41:54,18.9N,103.4W,36.1,15.6,-9.5,12.2,1.9,1.510000e+11,0.440
2,2,2023-07-07 16:15:31,14.5N,126.6W,38.3,25.7,2.6,4.1,-25.2,2.600000e+10,0.092
3,4,2023-06-21 19:39:13,12.4N,62.8E,40.4,16.5,7.3,-2.0,-16.4,5.430000e+11,1.400
4,5,2023-05-20 11:22:23,17.8S,141.9E,29.0,27.9,25.4,6.9,9.3,3.584000e+12,7.200
...,...,...,...,...,...,...,...,...,...,...,...
288,689,2004-04-22 04:21:49,44.3S,83.3E,29.6,16.3,8.5,-12.1,-6.8,6.240000e+11,1.500
289,690,2004-04-06 03:24:49,12.0S,162.8E,35.0,25.6,10.5,-23.2,-2.9,4.330000e+11,1.100
290,701,2003-11-10 13:54:06,64.5S,136.2E,23.0,20.1,14.8,-8.7,10.4,5.180000e+11,1.300
291,706,2003-09-27 12:59:02,21.0N,86.6E,26.0,18.2,-1.0,-5.4,-17.3,2.150000e+12,4.600


In [72]:
neos = pd.read_csv("datos/sbdb_query_results_NEOS.csv")
neos = neos[neos['a'] < 2]
neos

,pdes,epoch_mjd,a,e,i,om,w,ma
0,433,60200,1.458,0.2228,10.83,304.29,178.91,222.76
4,1221,60200,1.919,0.4357,11.88,171.32,26.65,123.53
5,1566,60200,1.078,0.8269,22.80,87.95,31.44,168.65
7,1620,60200,1.246,0.3355,13.34,337.17,276.95,5.84
8,1627,60200,1.864,0.3964,8.45,133.12,167.81,1.48
...,...,...,...,...,...,...,...,...
34320,2024 CM5,60200,1.867,0.5245,1.81,322.91,135.06,313.89
34321,2024 CN5,60353,1.623,0.3144,2.42,172.22,343.84,353.54
34322,2024 CO5,60352,1.708,0.3938,9.66,334.87,160.57,3.52
34325,2024 CR5,60352,1.136,0.3680,17.82,141.95,100.20,302.45


In [73]:
a = neos["a"]
e = neos["e"]
i = neos["i"]

fig = go.Figure(data=go.Splom(
                dimensions=[dict(label='a',
                                 values=a),
                            dict(label='e',
                                 values=e),
                            dict(label='i',
                                 values=i)],
                showupperhalf=False, # remove plots on diagonal
                diagonal_visible=False,
                opacity=0.5,
                marker=dict(showscale=False, # colors encode categorical variables
                            color='blue', size=0.7)
                ))


fig.update_layout(
    width=600,
    height=600
)

fig.show()

In [74]:
def change_coord(x):
    if x[-1] == 'N' or x[-1] == 'E':
        new = float(x[:-1])
    elif x[-1] == 'S' or x[-1] == 'W':
        new = -float(x[:-1])
    return new    

In [75]:
fireballs['Longitude deg'] = fireballs['Longitude (deg.)'].apply(change_coord)
fireballs['Latitude deg'] = fireballs['Latitude (deg.)'].apply(change_coord)

In [76]:
path = 'datos/kernels/'
spy.furnsh([path + 'naif0012.tls', path + 'pck00010.tpc', path + 'earth_fixed.tf', path + 'earth_720101_230601.bpc', path + 'earth_latest_high_prec.bpc'])

In [77]:
def transform_ecliptic(lon, lat, alt, date, vx, vy, vz):
    r = Geo2Rec(lon, lat, alt) 
    r_eclip = Geo2Eclip(lon, lat, alt, date, frame='IAU_EARTH')

    v = np.array([vx, vy, vz])  

    T_earth = 86400  
    omega = np.array([0,0,(2*np.pi)/T_earth]) 

    v_E = (v - spy.vcrss(omega, r))
    #v_E, -v, mag(v_E), np.arccos((v@r_irtf)/(np.linalg.norm(v)*np.linalg.norm(r_irtf)))*180/np.pi

    et = spy.utc2et(date)
    mx = spy.pxform('IAU_EARTH', 'ECLIPJ2000', et)
    v_eclip = spy.mxv(mx, v_E)

    return r_eclip, v_eclip

In [79]:
deg = 180/np.pi
AU = 149597870 #km
N = len(fireballs[:20])
orbit_elements = np.zeros((N, 6))
asteroid_statev = np.zeros((N, 6))
#print("time initial: ", time.time())
for bolid in range(N):
    lon = fireballs['Longitude deg'][bolid]
    lat = fireballs['Latitude deg'][bolid]
    alt = fireballs['Altitude (km)'][bolid]
    date = fireballs['Peak Brightness Date/Time (UT)'][bolid]
    vx = fireballs['vx'][bolid]
    vy = fireballs['vy'][bolid]
    vz = fireballs['vz'][bolid]

    r_eclip, v_eclip = transform_ecliptic(lon, lat, alt, date, vx, vy, vz)

    rb.horizons.SSL_CONTEXT = 'unverified'

    sim = rb.Simulation()
    sim.units = 'km', 's', 'kg'
    sim.integrator = "WHFast"
    sim.dt = 86400
    time = 3.154e+7

    for body in ["Sun", "Earth", "Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"]:
        sim.add(body, hash=body, date=date)

    sun = sim.particles[0]
    r_earth = np.array(sim.particles['Earth'].xyz)
    v_earth = np.array(sim.particles['Earth'].vxyz)

    r_asteroid = r_eclip  + r_earth 
    v_asteroid = v_eclip + v_earth #así si es 

    asteroid = sim.add(x=r_asteroid[0], y=r_asteroid[1], z=r_asteroid[2], 
                    vx=v_asteroid[0], vy=v_asteroid[1], vz=v_asteroid[2])

    #inicio integracion
    sim.integrate(-time)
    sim.move_to_com()
    asteroid_statev[bolid] = sim.particles[-1].xyz + sim.particles[-1].vxyz
    o = sim.orbits(primary=sun)[-1]
    orbit_elements[bolid] = [o.a/AU, o.e, o.inc*deg, o.Omega*deg, o.omega*deg, o.f] 

#print("time final: ", time.time())    

Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for 'Earth'... 
Found: Earth-Moon Barycenter (3) (chosen from query 'Earth')
Searching NASA Horizons for 'Mercury'... 
Found: Mercury Barycenter (199) (chosen from query 'Mercury')
Searching NASA Horizons for 'Venus'... 
Found: Venus Barycenter (299) (chosen from query 'Venus')
Searching NASA Horizons for 'Earth'... 
Found: Earth-Moon Barycenter (3) (chosen from query 'Earth')
Searching NASA Horizons for 'Mars'... 
Found: Mars Barycenter (4) (chosen from query 'Mars')
Searching NASA Horizons for 'Jupiter'... 
Found: Jupiter Barycenter (5) (chosen from query 'Jupiter')
Searching NASA Horizons for 'Saturn'... 
Found: Saturn Barycenter (6) (chosen from query 'Saturn')
Searching NASA Horizons for 'Uranus'... 
Found: Uranus Barycenter (7) (chosen from query 'Uranus')
Searching NASA Horizons for 'Neptune'... 
Found: Neptune Barycenter (8) (chosen from query 'Neptune')
Searching NASA Horizons for 'Sun'... 
Found: S

In [80]:
orbit_elements

array([[ 1.78174783e+01,  9.57587781e-01,  4.26836098e+00,
         1.32928296e+02,  1.18554218e+02,  3.94207952e+00],
       [ 1.23064518e+00,  5.08692848e-01,  6.53940830e+00,
         1.22660364e+02,  2.81526225e+02,  1.99319595e+00],
       [ 1.45180296e+00,  4.58535651e-01,  4.33525797e+01,
         1.04998475e+02,  1.06157163e+02,  3.16788154e+00],
       [ 8.70163858e-01,  3.26357548e-01,  3.40978209e+01,
         8.98781766e+01,  4.39301183e+01,  7.52287743e-01],
       [ 2.98343508e+00,  8.55993317e-01,  2.21056086e+00,
         5.89500210e+01,  2.84270637e+02,  3.45795309e+00],
       [ 6.30055204e+00,  8.60569376e-01,  6.47835814e+00,
        -1.44289693e+02,  3.16431327e+02,  3.94004050e+00],
       [ 3.68954684e+00,  7.48621237e-01,  2.08884253e+01,
        -1.55203678e+02,  3.25279502e+02,  3.86430150e+00],
       [ 2.17241026e+00,  5.40271336e-01,  3.66044456e+01,
         1.62274749e+01,  1.86299775e+02,  3.58012477e+00],
       [ 7.83905715e-01,  8.52253942e-01,  1.445

In [81]:
a_int = orbit_elements[:,0]
a_int = a_int[(a_int > 0) & (a_int < 2)]
e_int = orbit_elements[:,1]
e_int = e_int[(e_int > 0) & (e_int < 1)]
i_int = orbit_elements[:,2]

fig = go.Figure()

# Agregar el scatter plot principal
fig.add_trace(go.Splom(
    dimensions=[dict(label='a', values=a),
                dict(label='e', values=e),
                dict(label='i', values=i)],
    showupperhalf=False,
    diagonal_visible=False,
    opacity=0.5,
    marker=dict(showscale=False, color='blue', size=0.7)
))

# Agregar los 10 puntos en rojo
fig.add_trace(go.Splom(
    dimensions=[dict(label='a', values=a_int),
                dict(label='e', values=e_int),
                dict(label='i', values=i_int)],
    showupperhalf=False,
    diagonal_visible=False,
    opacity=1.0,  # Puntos completamente visibles
    marker=dict(showscale=False, color='red', size=3)  # Rojo y más grandes
))

# Ajustar el layout
fig.update_layout(
    width=600,
    height=600
)

fig.show()

## Get SSB positions for all planets 

In [ ]:
sun = np.zeros((len(fireballs),6))
earth = np.zeros((len(fireballs),6))
mercury = np.zeros((len(fireballs),6))
venus = np.zeros((len(fireballs),6))
mars = np.zeros((len(fireballs),6))
jupiter = np.zeros((len(fireballs),6))
saturn = np.zeros((len(fireballs),6))
uranus = np.zeros((len(fireballs),6))
neptune = np.zeros((len(fireballs),6))

for i, date in enumerate(fireballs['Peak Brightness Date/Time (UT)']):
    start =  datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    stop = start + datetime.timedelta(minutes=1)

    sun = Horizons(id = '10', location='@0', epochs = dict(start = str(start), stop = str(stop), step = '1m')).vectors()
    earth = Horizons(id = '399', location='@0', epochs = dict(start = str(start), stop = str(stop), step = '1m')).vectors()
    sun = Horizons(id = '10', location='@0', epochs = dict(start = str(start), stop = str(stop), step = '1m')).vectors()
    earth = Horizons(id = '399', location='@0', epochs = dict(start = str(start), stop = str(stop), step = '1m')).vectors()
    sun = Horizons(id = '10', location='@0', epochs = dict(start = str(start), stop = str(stop), step = '1m')).vectors()
    earth = Horizons(id = '399', location='@0', epochs = dict(start = str(start), stop = str(stop), step = '1m')).vectors()

    earth[i] = [earth[0]['x'], earth[0]['y'],earth[0]['z']]
    sun[i] = [sun[0]['x'], sun[0]['y'], sun[0]['z']]
